This tutorial will apply the knowledge from the previous `"001.cloud-native-format-access.ipynb"`. We are going to retrieve OLM-GEDI in the area of Benelux, and then we will overlay with some auxiliary raster layers, such as land cover, DTM height, canopy height models, surface water, and so on. By retrieval of the pixel value, we are able to fliter some outliers, and conduct stratification data table for better representation for the next coming canopy height modeling.

In [1]:
# Install required packages
## Install Eigen, and other system libraries
!apt-get update
!apt-get install -y libeigen3-dev

## Install recent version of pybind11
!sudo apt-get install cmake
!git clone https://github.com/pybind/pybind11.git
%cd pybind11
!git checkout v2.13.6
!mkdir build
%cd build
!cmake ..
!make -j$(nproc --all)
!sudo make install

## Create symbolic links for Eigen (this step may not always be needed)
!ln -sf /usr/include/eigen3/Eigen /usr/include/Eigen
!ln -sf /usr/include/eigen3/unsupported /usr/include/unsupported
## Install Eigen, and other system libraries
!apt-get update
!apt-get install -y libeigen3-dev

## Install recent version of pybind11
!sudo apt-get install cmake
!git clone https://github.com/pybind/pybind11.git
%cd pybind11
!git checkout v2.13.6
!mkdir build
!apt-get install -y libproj-dev libgeos-dev gdal-bin libgdal-dev postgis

## Install scikit-map
!pip install git+https://github.com/openlandmap/scikit-map.git@develop

## Install PROJ, GEOS, GDAL, and PostGIS-related libraries
!apt-get install -y libproj-dev libgeos-dev gdal-bin libgdal-dev postgis

## Install scikit-map
!pip install git+https://github.com/openlandmap/scikit-map.git@develop

!pip install minio

import os
## !find /usr -name proj.db
## !pip install rasterio pyproj
os.unsetenv('PROJ_LIB')
os.environ['PROJ_LIB'] = '/usr/local/lib/python3.11/dist-packages/rasterio/proj_data'


!pip install -q gdown
%cd /content

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [2]:
# load in GEDI02 stac items
import geopandas as gpd
gedi_items=gpd.read_file('https://s3.eu-central-1.wasabisys.com/stac/openlandmap/GEDI02/stac_items_geojson')
print(gedi_items.columns)

Index(['item_id', 'size_in_mb', 'point_counts', 'year', 'start_date',
       'end_date', 'platform', 'stac_extentsion', 'asset_file', 'geometry'],
      dtype='object')


In [3]:
# load in the Dutch borders by GADM
! wget https://geodata.ucdavis.edu/gadm/gadm4.1/json/gadm41_NLD_1.json.zip
! unzip gadm41_NLD_1.json.zip

--2025-08-29 16:24:53--  https://geodata.ucdavis.edu/gadm/gadm4.1/json/gadm41_NLD_1.json.zip
Resolving geodata.ucdavis.edu (geodata.ucdavis.edu)... 128.120.146.30
Connecting to geodata.ucdavis.edu (geodata.ucdavis.edu)|128.120.146.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29252 (29K) [application/zip]
Saving to: ‘gadm41_NLD_1.json.zip.2’

gadm41_NLD_1.json.z 100%[===================>]  28.57K  --.-KB/s    in 0.07s   

2025-08-29 16:24:54 (381 KB/s) - ‘gadm41_NLD_1.json.zip.2’ saved [29252/29252]

Archive:  gadm41_NLD_1.json.zip
replace gadm41_NLD_1.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: gadm41_NLD_1.json       


In [4]:
nl_level_1 = 'gadm41_NLD_1.json'
nl_provinces = gpd.read_file(nl_level_1)
print('crs of nl provinces -',nl_provinces.crs)
nl_provinces.head()

crs of nl provinces - EPSG:4326


,GID_1,GID_0,COUNTRY,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,ISO_1,geometry
0,NLD.1_1,NLD,Netherlands,Drenthe,NA,NA,Provincie,Province,NA,NL.DR,NL-DR,"MULTIPOLYGON (((6.5237 52.6455, 6.5274 52.6148..."
1,NLD.2_1,NLD,Netherlands,Flevoland,NA,NA,Provincie,Province,NA,NL.FL,NL-FL,"MULTIPOLYGON (((5.3039 52.3125, 5.245 52.3294,..."
2,NLD.3_1,NLD,Netherlands,Fryslân,Friesland,NA,Provincie,Province,NA,NL.FR,NL-FR,"MULTIPOLYGON (((6.2392 52.9132, 6.2149 52.8909..."
3,NLD.4_1,NLD,Netherlands,Gelderland,Geldern|Gheldria|Guelders|Gueldr,NA,Provincie,Province,NA,NL.GE,NL-GE,"MULTIPOLYGON (((5.1358 51.7385, 5.1314 51.7394..."
4,NLD.5_1,NLD,Netherlands,Groningen,Groninga|Groningue,NA,Provincie,Province,NA,NL.GR,NL-GR,"MULTIPOLYGON (((6.9171 53.0116, 6.7468 53.1189..."


In [5]:
from shapely.geometry import box
## Narrow down GEDI data
# 1. Get the total bounding box of Dutch borders
minx, miny, maxx, maxy = nl_provinces.total_bounds
bbox = box(minx, miny, maxx, maxy)

# 2. Filter GEDI STAC items  by intersection with bbox
gedi_nl = gedi_items[gedi_items.intersects(bbox)]

In [6]:
# get all the urls from the target stac items
urls=[i for i in gedi_nl.asset_file]
print(urls)

['https://s3.opengeohub.org/global/glidar/gedi-ard/level2/l2v002.gedi_20190418_20230316_go_epsg.4326_v20240827/lon=0/lat=50/year=2019/gedi_l2ab.parquet', 'https://s3.opengeohub.org/global/glidar/gedi-ard/level2/l2v002.gedi_20190418_20230316_go_epsg.4326_v20240827/lon=0/lat=50/year=2020/gedi_l2ab.parquet', 'https://s3.opengeohub.org/global/glidar/gedi-ard/level2/l2v002.gedi_20190418_20230316_go_epsg.4326_v20240827/lon=0/lat=50/year=2021/gedi_l2ab.parquet', 'https://s3.opengeohub.org/global/glidar/gedi-ard/level2/l2v002.gedi_20190418_20230316_go_epsg.4326_v20240827/lon=0/lat=50/year=2022/gedi_l2ab.parquet', 'https://s3.opengeohub.org/global/glidar/gedi-ard/level2/l2v002.gedi_20190418_20230316_go_epsg.4326_v20240827/lon=0/lat=50/year=2023/gedi_l2ab.parquet', 'https://s3.opengeohub.org/global/glidar/gedi-ard/level2/l2v002.gedi_20190418_20230316_go_epsg.4326_v20240827/lon=5/lat=50/year=2019/gedi_l2ab.parquet', 'https://s3.opengeohub.org/global/glidar/gedi-ard/level2/l2v002.gedi_20190418_202

In [7]:
# Retrieve OLM-GEDI where
## - data falls within overlapped OLM-GEDI tiles
## - sensitivity is greater than 97 (scale = 100)
import duckdb
import time
start_time  = time.time()
urls=[i for i in gedi_nl.asset_file]
df_duckdb = duckdb.sql(f"""
                        INSTALL httpfs;
                        LOAD httpfs;
                        INSTALL spatial;
                        LOAD spatial;
                        SELECT rh98, latitude, longitude
                        FROM read_parquet({urls})
                        WHERE longitude BETWEEN {minx} AND {maxx}
                            AND latitude BETWEEN {miny} AND {maxy}
                            AND sensitivity > 9700;

                        """)
gedi_points = df_duckdb.df()
print(f"--- %s seconds ---" % (time.time() - start_time))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

--- 41.33479571342468 seconds ---


In [8]:
# Convert to GeoPandas and export random 5000 points for inspection
gdf = gpd.GeoDataFrame(
    gedi_points, geometry=gpd.points_from_xy(gedi_points['longitude'], gedi_points['latitude']))
gdf.sample(5000).to_file('gedi_sample500_nl.geojson')

/usr/local/lib/python3.12/dist-packages/pyogrio/geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


In [10]:
# Download overlay raster layer information
! wget https://s3.eu-central-1.wasabisys.com/ogh/faen/covs_nl_canopy_height.csv

--2025-08-29 16:32:50--  https://s3.eu-central-1.wasabisys.com/ogh/faen/covs_nl_canopy_height.csv
Resolving s3.eu-central-1.wasabisys.com (s3.eu-central-1.wasabisys.com)... 130.117.252.103, 130.117.252.107, 130.117.252.102, ...
Connecting to s3.eu-central-1.wasabisys.com (s3.eu-central-1.wasabisys.com)|130.117.252.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1736 (1.7K) [text/csv]
Saving to: ‘covs_nl_canopy_height.csv.2’

covs_nl_canopy_heig 100%[===================>]   1.70K  --.-KB/s    in 0s      

2025-08-29 16:32:51 (560 MB/s) - ‘covs_nl_canopy_height.csv.2’ saved [1736/1736]



In [11]:
# Create the catalog object from the csv for auxiliary data overlay
from skmap.catalog import DataCatalog
from minio import Minio
catalog_path = 'covs_nl_canopy_height.csv'
base_path = 'https://s3.opengeohub.org'
json_out_path = 'nl_canopy_height.json'
catalog = DataCatalog.create_catalog(catalog_def=catalog_path, years=[], base_path=base_path)
catalog.save_json(json_out_path)

In [12]:
# Inpect the catalog, and create thread number info to match the parallel optimization
n_threads=len(catalog.data['common'])
print(catalog.data)

{'common': {'gedtm_rf_m_30m_s_20060101_20151231_go_epsg.4326.3855_v20250611': {'path': 'https://s3.opengeohub.org/global/edtm/gedtm_rf_m_30m_s_20060101_20151231_go_epsg.4326.3855_v20250611.tif', 'idx': 0}, 'geoid_eigen_gfz': {'path': 'https://s3.opengeohub.org/global/edtm/geoid_eigen_gfz.tif', 'idx': 1}, 'surface.water_jrc.gswe_p_30m_s_2000_2018_go_epsg.4326_v01232022': {'path': 'https://s3.opengeohub.org/global/edtm/surface.water_jrc.gswe_p_30m_s_2000_2018_go_epsg.4326_v01232022.tif', 'idx': 2}, 'lc_glad.glcluc.change_c_30m_s_20000101_20201231_go_epsg.4326_v20230901': {'path': 'https://s3.opengeohub.org/global/lc/lc_glad.glcluc.change_c_30m_s_20000101_20201231_go_epsg.4326_v20230901.tif', 'idx': 3}, 'canopy.height_glad.umd_m_30m_s_2017_2018_go_epsg.4326_v01242022': {'path': 'https://s3.opengeohub.org/global/edtm/canopy.height_glad.umd_m_30m_s_2017_2018_go_epsg.4326_v01242022.tif', 'idx': 4}, 'canopy.height_eth_m_30m_s_20000101_20230101_go_epsg.4326_v01242022': {'path': 'https://s3.ope

In [13]:
GDAL_OPTS = {'GDAL_HTTP_VERSION': '1.0', 'CPL_VSIL_CURL_ALLOWED_EXTENSIONS': '.tif'}
max_ram_mb = 1200

In [2]:
# Run the overlay
from skmap.overlay import SpaceOverlay
start = time.time()
space_overlay = SpaceOverlay(
        points=gdf,
        catalog=catalog,
        verbose=True,
        n_threads=n_threads)
print(f"Extraction of overlay meta-data: {(time.time() - start):.2f} s")
start = time.time()
ovelayed_data = space_overlay.run(gdal_opts=GDAL_OPTS, max_ram_mb=max_ram_mb, out_file_name="overlay_gedi.pq")
print(f"Reading overlayed layers: {(time.time() - start):.2f} s")

NameError: name 'time' is not defined

In [1]:
# Inspection on overlaid data
ovelayed_data.head()
print(ovelayed_data.columns)

NameError: name 'ovelayed_data' is not defined

In [ ]:
# Filtering of overlayed_data
### 1. remove gedi point that has possibility to fall on water surface (https://global-surface-water.appspot.com/)
o1=ovelayed_data[ovelayed_data['surface.water_jrc.gswe_p_30m_s_2000_2018_go_epsg.4326_v01232022']==0]

In [ ]:
# Inspetion on GEDI terrain hieght with associated DTM model
gedi_dtm=o1.elev_lowestmode*0.01 - o1.geoid_eigen_gfz # correction to EGM2008 (https://dataservices.gfz-potsdam.de/icgem/showshort.php?id=escidoc:1119897)
edtm=o1['gedtm_rf_m_30m_s_20060101_20151231_go_epsg.4326.3855_v20250611']*0.1 # rescale EDTM from dm to m
import matplotlib.pyplot as plt
plt.scatter(
    gedi_dtm, edtm,
    s=60,                  # point size
    c="dodgerblue",        # point color
    edgecolors="black",    # outline
    alpha=0.7              # transparency
)

plt.title("Scatterplot of Points in the Netherlands", fontsize=14, weight="bold")
plt.xlabel("GEDI terrain height", fontsize=12)
plt.ylabel("EDTM terrain height", fontsize=12)

plt.grid(True, linestyle="--", alpha=0.5)
plt.show()

In [ ]:
# Filtering of overlayed_data
### 2. remove gedi point where it has 20m difference from the terrain height (GEDTM: https://peerj.com/articles/19673/)
o2 = o1[abs(gedi_dtm-edtm)<20]
o2_gedi = o2.elev_lowestmode*0.01 - o2.geoid_eigen_gfz
o2_edtm = o2['gedtm_rf_m_30m_s_20060101_20151231_go_epsg.4326.3855_v20250611']*0.1

In [ ]:
# Inspetion on GEDI terrain hieght with associated DTM model after filtering
plt.scatter(
    o2_gedi, o2_edtm,
    s=60,                  # point size
    c="dodgerblue",        # point color
    edgecolors="black",    # outline
    alpha=0.7              # transparency
)

plt.title("Scatterplot of Points in the Netherlands", fontsize=14, weight="bold")
plt.xlabel("GEDI terrain height", fontsize=12)
plt.ylabel("EDTM terrain height", fontsize=12)

plt.grid(True, linestyle="--", alpha=0.5)
plt.show()

In [ ]:
# Filtering of overlayed_data
### 2. remove non-canpoy pixel
import numpy as np
o3=o2[~np.isnan(o2['canopy.height_eth_m_30m_s_20000101_20230101_go_epsg.4326_v01242022'])] # no canopy height info at ETH canopy height (https://gee-community-catalog.org/projects/canopy/)
o4=o3[o3['lc_esa.worldcover_c_10m_s_20210101_20211231_go_epsg.4326_v20240901']==10] # not forest at ESA world cover (https://esa-worldcover.org/)

In [ ]:
# Stratification by tree cover and canopy height
### - tree cover 0-10, 10-20, ..., 90-100
### - tree height 0-5, 5-10, 10-15,... 45-50

## 1. Create tree cover strata (10% bins)
tree_bins = range(0, 101, 10)  # 0-10, 10-20, ...
o4["tree_stratum"] = pd.cut(o4["tree.cover_glad.umd_m_30m_s_20100101_20151231_go_epsg.4326_v1.0"], bins=tree_bins, right=False)

## 2. Create canopy height strata (5m bins)
height_bins = range(0, 51, 5)  # 0-5, 5-10, ...
o4["height_stratum"] = pd.cut(o4["canopy.height_eth_m_30m_s_20000101_20230101_go_epsg.4326_v01242022"], bins=height_bins, right=False)

## 3. Combine both into one stratification column
o4["stratum"] = o4["tree_stratum"].astype(str) + " | " + o4["height_stratum"].astype(str)

In [ ]:
## 4.Count the possible strata sample number
stratified_counts = (
    o4.groupby("stratum", group_keys=False)
      .apply(lambda x: len(x))
      .reset_index(drop=True))

In [ ]:
# Visualize the strata count distribution
plt.hist(stratified_counts,bins=100)

In [ ]:
# Assign a cap for stratum sample number
n_per_stratum=int(np.median(stratified_counts))
n_per_stratum

In [ ]:
# Stratification
stratified_sample = (
    o4.groupby("stratum", group_keys=False)
      .apply(lambda x: x.sample(n_per_stratum, random_state=1) if len(x)>n_per_stratum else x)
      .reset_index(drop=True)
)

In [ ]:
# Inspection on strata
stratified_sample.groupby("stratum", group_keys=False).count()

In [ ]:
# Wrap up to GeoPandas and export to GeoJSON
gdf=gpd.GeoDataFrame(
    stratified_sample, geometry=gpd.points_from_xy(stratified_sample['longitude'], stratified_sample['latitude']))
gdf[['rh98','longitude','latitude','geometry']].to_file('gedi_can_sampling.geojson')

In [ ]:
# Create the catalog object from the csv for embedding overlay
from skmap.catalog import DataCatalog
from minio import Minio
catalog_path = 'embedding.csv'
base_path = 'https://s3.opengeohub.org'
json_out_path = 'embedding.json'
catalog = DataCatalog.create_catalog(catalog_def=catalog_path, years=[], base_path=base_path)
catalog.save_json(json_out_path)

In [ ]:
# Run the overlay
from skmap.overlay import SpaceOverlay
start = time.time()
space_overlay = SpaceOverlay(
        points=gdf,
        catalog=catalog,
        verbose=True,
        n_threads=n_threads)
print(f"Extraction of overlay meta-data: {(time.time() - start):.2f} s")
start = time.time()
ovelayed_data = space_overlay.run(gdal_opts=GDAL_OPTS, max_ram_mb=max_ram_mb, out_file_name="overlay_gedi.pq")
print(f"Reading overlayed layers: {(time.time() - start):.2f} s")

In [ ]:
ovelayed_data.to_csv('nl_embedding_gedi_canopy_height.csv')